In [1]:
import os
from pathlib import Path
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import shutil

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
dataroot = Path("/media/smbshare/srs-9/prl_project/training/test_train3")
training_work_home = Path("/home/srs-9/Projects/prl_project/training_work_dirs")
work_dir = training_work_home / "test_train3_segresnet" #! edit this
datalist_file_src = os.path.join(os.getcwd(), "datalist.json")
datalist_file = work_dir / "datalist.json"
algos = ["segresnet"]

description = """Model: SegResNet
Trained using flair.phase.t1 images. 
Labels by Lia, edited to make 5->1, 4->1 (extra T2L and iron content as red) and overlaid on lst-ai segs"""

Check that every file in the datalist exists

In [3]:
import json

with open(datalist_file_src, 'r') as f:
    datalist = json.load(f)

for item in datalist['training']:
    assert os.path.exists(item['image'])
    assert os.path.exists(item['label'])

for item in datalist['testing']:
    assert os.path.exists(item['image'])

Create the work_dir, copy over the datalist, and save an info file

In [18]:
if not work_dir.exists():
    print(f"Creating {str(work_dir)}")
    os.makedirs(work_dir)

shutil.copyfile(datalist_file_src, datalist_file)

with open(work_dir / "info.txt", 'w') as f:
    f.write(description)

print("work_dir is: ", work_dir)

work_dir is:  /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet


In [4]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=algos,
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2025-08-21 17:09:18,738 - INFO - AutoRunner using work directory /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet
2025-08-21 17:09:18,756 - INFO - Found num_fold 2 based on the input datalist /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/datalist.json.
2025-08-21 17:09:18,756 - INFO - Setting num_fold 2 based on the input datalist /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/datalist.json.
2025-08-21 17:09:18,773 - INFO - Using user defined command running prefix , will override other settings


In [5]:
max_epochs = 250

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [6]:
runner.run()

2025-08-21 17:09:26,078 - INFO - Skipping data analysis...
2025-08-21 17:09:26,079 - INFO - Skipping algorithm generation...
2025-08-21 17:09:26,088 - INFO - ['python', '/home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/segresnet_0/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/segresnet_0/configs/hyper_parameters.yaml'", '--num_epochs_per_validation=1', '--num_epochs=250', '--num_warmup_epochs=1']


`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
segresnet_0 - training:   0%|          | 0/250 [00:00<?, ?epoch/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
segresnet_0 - training: 100%|██████████| 250/250 [52:24<00:00, 12.58s/epoch]


2025-08-21 18:01:56,612 - INFO - ['python', '/home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/segresnet_1/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/segresnet_1/configs/hyper_parameters.yaml'", '--num_epochs_per_validation=1', '--num_epochs=250', '--num_warmup_epochs=1']


`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/08/21 18:02:00 INFO mlflow.tracking.fluent: Experiment with name 'Auto3DSeg' does not exist. Creating a new experiment.
segresnet_1 - training:   0%|          | 0/250 [00:00<?, ?epoch/s]`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
segresnet_1 - training: 100%|██████████| 250/250 [1:01:17<00:00, 14.71s/epoch]


2025-08-21 19:03:19,986 - INFO - Ensembling using single GPU!
2025-08-21 19:03:19,986 - INFO - The output_dir is not specified. /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/ensemble_output will be used to save ensemble predictions.
2025-08-21 19:03:19,986 - INFO - Directory /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/ensemble_output is created to save ensemble predictions
2025-08-21 19:03:20,018 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-08-21 19:03:20,018 - INFO - segresnet_0
2025-08-21 19:03:20,019 - INFO - segresnet_1
2025-08-21 19:03:20,019 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/prl_project/training_work_dirs/test_train3_segresnet/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/3 [00:00<?, ?it/s]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScale